In [9]:
import numpy as np
import pandas as pd

dataset = pd.read_csv('../datasets/T20-Edit - Sheet1.csv')

In [83]:
# v= dataset.batting_team.unique()
# # v

# d = pd.read_csv('../datasets/T20-Edit - Sheet1.csv')
# d[d['venue'] == 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium']

In [84]:
full_data = {}
y = []
unique_match_ids = dataset.match_id.unique()


for matchId in unique_match_ids:
    if matchId not in full_data.keys():
        data = dataset[(dataset['match_id'] == matchId)][['batting_team']]
        full_data[matchId] = data['batting_team'].unique()
        
for key in full_data.keys():
    team1 = dataset.loc[(dataset['match_id'] == key) & (dataset['batting_team'] == full_data[key][0])][['runs_off_bat']].sum().to_numpy()[0] + dataset.loc[(dataset['match_id'] == key) & (dataset['batting_team'] == full_data[key][0])][['extras']].sum().to_numpy()[0]
    team2 = dataset.loc[(dataset['match_id'] == key) & (dataset['batting_team'] == full_data[key][1])][['runs_off_bat']].sum().to_numpy()[0] + dataset.loc[(dataset['match_id'] == key) & (dataset['batting_team'] == full_data[key][1])][['extras']].sum().to_numpy()[0]
    data = dataset[(dataset['match_id'] == key) & (dataset['batting_team'] == full_data[key][0])]
    data2 = dataset[(dataset['match_id'] == key) & (dataset['batting_team'] == full_data[key][1])]
    
    data = int(len(data['batting_team']))
    data2 = int(len(data2['batting_team']))
    r_list = np.tile(np.array(team1), data) 
    r2_list = np.tile(np.array(team2), data2)
    y = list(y) + list(r_list) + list(r2_list)
    
dataset['y'] = y
dataset.shape

(146669, 17)

In [85]:
dataset = dataset.drop(['match_id', 'start_date', 'player_dismissed', 'wicket_type', 'extras', 'runs_off_bat','innings',
                        'ball', 'striker', 'non_striker', 'bowler','season'],axis=1)

In [86]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [87]:
# venue_pitch_cond = {"batting_pitches" : [], "bowling_pitches": []}

# for i in dataset.venue.unique():
#     if len(dataset.loc[(dataset['venue'] == i) & (dataset.pitch_conditon == 'batting')]) != 0:
#         venue_pitch_cond['batting_pitches'].append(i)
#     elif len(dataset.loc[(dataset['venue'] == i) & (dataset.pitch_conditon == 'bowling')]) != 0:
#         venue_pitch_cond['bowling_pitches'].append(i)

# venue_pitch_cond

In [88]:
le_venue = LabelEncoder()
le_pitch_cond = LabelEncoder()
le_batting_team = LabelEncoder()
le_bowling_team = LabelEncoder()

In [89]:
dataset['venue'] = le_venue.fit_transform(dataset['venue'])
dataset['pitch_conditon'] = le_pitch_cond.fit_transform(dataset['pitch_conditon'])
dataset['batting_team'] = le_batting_team.fit_transform(dataset['batting_team'])
dataset['bowling_team'] = le_bowling_team.fit_transform(dataset['bowling_team'])

In [90]:
X = dataset.drop(['y'],axis=1)
Y = dataset['y']

In [91]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(X,Y, test_size = 0.33)

In [18]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(Xtrain, Ytrain)

linear_reg_predict = linear_reg.predict(Xtest)
print(linear_reg.score(Xtest,Ytest))

0.004520375203685756


In [19]:
from sklearn.naive_bayes import GaussianNB

naive_bayes_algo = GaussianNB()

naive_bayes_algo.fit(Xtrain,Ytrain)

y_pred_nb = naive_bayes_algo.predict(Xtest)

print(accuracy_score(Ytest,y_pred_nb))

0.03961273607418013


In [20]:
from sklearn.ensemble import RandomForestClassifier

random_forest_class  = RandomForestClassifier(n_estimators = 100)

random_forest_class.fit(Xtrain,Ytrain)

y_pred_rfc = random_forest_class.predict(Xtest)

print(accuracy_score(Ytest,y_pred_rfc))

0.6995068293902411


In [21]:
from sklearn.ensemble import RandomForestRegressor

random_f_reg = RandomForestRegressor(max_depth=10)

random_f_reg.fit(Xtrain,Ytrain)

y_pred_rfr = random_f_reg.predict(Xtest)

In [22]:
print(random_f_reg.score(Xtrain,Ytrain))

0.4728924771673215


In [92]:
from sklearn.tree import DecisionTreeRegressor

dec_tree_reg = DecisionTreeRegressor()

dec_tree_reg.fit(Xtrain,Ytrain)

# xyz = np.array([['Tolerance Oval', 'batting', 'Australia', 'India']])
# xyz[:,0] = le_venue.fit_transform(xyz[:,0])
# xyz[:,1] = le_pitch_cond.fit_transform(xyz[:,1])
# xyz[:,2] = le_team.fit_transform(xyz[:, 2])
# xyz[:,3] = le_team.fit_transform(xyz[:, 3])
y_pred_dtr = dec_tree_reg.predict(Xtest)

print(dec_tree_reg.score(Xtrain,Ytrain))
print(y_pred_dtr)


0.818123433063287
[140.          96.22119816 142.         ...  81.         121.17857143
 191.68789809]


In [94]:
import pickle

crick_pred = {'model': dec_tree_reg, 'le_venue':le_venue, 'le_pitch_cond':le_pitch_cond,'le_batting_team': le_batting_team,
             'le_bowling_team': le_bowling_team
             }

with open('cricket_pred.pkl', 'wb') as file:
    pickle.dump(crick_pred, file)

In [3]:
mens_data = pd.read_csv('../datasets/Men T20I Player Innings Stats - 21st Century.csv')
# mens_data.head()

In [4]:
unique_male_players = mens_data['Innings Player'].unique()
len(unique_male_players)

1080

In [5]:
t = {}

uni_t = dataset.batting_team.unique()

for i in uni_t:
    striker = dataset[(dataset['batting_team'] == i)]['striker'].unique()
    bowlers = dataset[dataset['bowling_team'] == i]['bowler'].unique()
    total = list(set(list(striker) + list(bowlers)))
    total = [i for i in total if i in unique_male_players]
    if len(total) >=11:
        t[i] = total

In [6]:
t

{'Australia': ['B Stanlake',
  'MC Henriques',
  'JR Hazlewood',
  'JP Behrendorff',
  'DJM Short',
  'MA Starc',
  'CA Lynn',
  'AT Carey',
  'AC Agar',
  'MS Wade',
  'AJ Finch',
  'DT Christian',
  'A Zampa',
  'JA Richardson',
  'TM Head',
  'AJ Turner',
  'KW Richardson',
  'PSP Handscomb',
  'PJ Cummins',
  'MJ Swepson',
  'DA Warner',
  'SA Abbott',
  'AJ Tye',
  'GJ Maxwell',
  'BR McDermott',
  'MP Stoinis',
  'MR Marsh',
  'SPD Smith'],
 'Ireland': ['AR McBrine',
  'HT Tector',
  'PKD Chase',
  'Simi Singh',
  "KJ O'Brien",
  'PR Stirling',
  'A Balbirnie',
  'GH Dockrell',
  'GJ Thompson',
  'SW Poynter',
  'BJ McCarthy',
  'MR Adair',
  'CA Young',
  'GJ Delany',
  'TE Kane',
  'SC Getkate',
  'WB Rankin',
  'GC Wilson',
  'DC Delany',
  'SR Thompson'],
 'Afghanistan': ['Usman Ghani',
  'Najibullah Zadran',
  'Sharafuddin Ashraf',
  'Hamid Hassan',
  'Rashid Khan',
  'Gulbadin Naib',
  'Shafiqullah',
  'Shapoor Zadran',
  'Mohammad Nabi',
  'Mohammad Shahzad',
  'Amir Hamza

In [8]:
f = {k: v for k,v in t.items() if v}
f.keys()

dict_keys(['Australia', 'Ireland', 'Afghanistan', 'India', 'Zimbabwe', 'Scotland', 'Netherlands', 'Namibia', 'Kenya', 'Sri Lanka', 'Pakistan', 'England', 'South Africa', 'Bangladesh', 'West Indies', 'New Zealand', 'Bermuda'])

In [15]:
len(dataset[dataset['batting_team'] == "India"]['bowler'].unique())

119

In [20]:
dff = dataset[(dataset['batting_team'] == "India") & (dataset['striker'] == "RG Sharma") & (dataset['start_date'] == "24-02-2019")
             & (dataset['batting_team'] == "India")].values.tolist()
dff[0]

[1168247,
 '2018/19',
 '24-02-2019',
 'Dr. Y.S. Rajasekhara Reddy ACA-VDCA Cricket Stadium',
 'bowling',
 1,
 0.1,
 'India',
 'Australia',
 'RG Sharma',
 'KL Rahul',
 'JP Behrendorff',
 0,
 0,
 '.',
 '.']